In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -q accelerate>=0.16.0 torchvision datasets diffusers  bitsandbytes

In [ ]:
!accelerate config default

In [ ]:
!git clone https://github.com/docty/diffuser-training.git

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

In [ ]:
import shutil
shutil.rmtree('/content/dog/.cache')

In [ ]:
!accelerate launch /content/diffuser-training/train_dreambooth_lora.py \
  --pretrained_model_name_or_path="stable-diffusion-v1-5/stable-diffusion-v1-5"  \
  --instance_data_dir="dog" \
  --output_dir="dreambooth-model-lora" \
  --instance_prompt="a photo of sks dog" \
  --resolution=128 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="A photo of sks dog in a bucket" \
  --validation_epochs=50 \
  --seed="0" \
  --push_to_hub

# Inference

In [ ]:
import torch
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5").to("cuda")
pipe.load_lora_weights("Docty/dreambooth-model-lora")


In [ ]:
prompt = "A picture of a sks dog in a bucket"
image = pipe(prompt, num_inference_steps=25).images[0]
#image.save("dog-bucket.png")
image

In [ ]:
from huggingface_hub.repocard import RepoCard
from diffusers import StableDiffusionPipeline
import torch

lora_model_id = "Docty/dreambooth-model-lora"
card = RepoCard.load(lora_model_id)
base_model_id = card.data.to_dict()["base_model"]

pipe = StableDiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.load_lora_weights(lora_model_id)

In [ ]:
image = pipe("A picture of a sks dog in a bucket", num_inference_steps=25).images[0]
image